In [4]:
source ../config_duke.sh; show_env

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc



In [5]:
chmod +x test.sh

In [11]:
X=10
./test.sh --env X=${X} -c a b c

--env
X=10
-c
a
b
c


In [2]:
cat ${FD_PRJ}/sing_proj_encode_fcc.sh

#!/bin/bash

### get file path
### Stackoverflow: how-can-i-get-the-source-directory-of-a-bash-script-from-within-the-script-itself
SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
source ${SCRIPT_DIR}/config_duke.sh
 
### execute
singularity exec \
    -H ${PWD}:/home \
    -B ${FD_WORK}:/mount/work \
    -B ${FD_PRJ}:/mount/project \
    -B ${FD_RLAB}:/mount/reddylab \
    "$-" \
    ${FD_SING}/singularity_proj_combeffect.sif "$@"



In [3]:
${FD_PRJ}/sing_proj_encode_fcc.sh ls

FATAL:   could not open image /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/scratch/hB: failed to retrieve path for /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/scratch/hB: lstat /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/scratch/hB: no such file or directory


: 255

In [6]:
${FD_PRJ}/sing_proj_encode_fcc.sh ls /mount/project/scratch

A.bed		      Sing_heredoc.ipynb	      mp_benchmark.py
B.bed		      add_starrseq_to_abc_copy.ipynb  search_alex_code.ipynb
R_bins_barplot.ipynb  bedtools_coverage.ipynb
R_slice_window.ipynb  mp_benchmark.ipynb


In [7]:
${FD_PRJ}/sing_proj_encode_fcc.sh ls *.bed

A.bed  B.bed


In [8]:
${FD_PRJ}/sing_proj_encode_fcc.sh ls /mount/project/scratch/*.bed

/bin/ls: cannot access '/mount/project/scratch/*.bed': No such file or directory


: 2

In [11]:
singularity exec -B ${FD_PRJ}:/mount/project ${FD_SING}/singularity_proj_combeffect.sif ls

Kuei	  config_proj_combeffect.sh  monitor.sh  tmp
Rlibs	  config_proj_encode_fcc.sh  reddylab	 watching.sh
config.R  module_avail.txt	     scratch


In [12]:
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash <<'EOF'
ls
EOF

A.bed		      Sing_heredoc.ipynb	      mp_benchmark.py
B.bed		      add_starrseq_to_abc_copy.ipynb  search_alex_code.ipynb
R_bins_barplot.ipynb  bedtools_coverage.ipynb
R_slice_window.ipynb  mp_benchmark.ipynb


In [13]:
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash <<'EOF'
ls /mount/project/scratch/*.bed
EOF

/mount/project/scratch/A.bed  /mount/project/scratch/B.bed


## Test nested heredoc

In [1]:
cat <<EOF1
Hello from level 1
    cat <<EOF2
    Hello from level 2
    EOF2
EOF1

Hello from level 1
    cat <<EOF2
    Hello from level 2
    EOF2


In [2]:
cat <<-EOF1
Hello from level 1
    cat <<-EOF2
    Hello from level 2
    EOF2
EOF1

Hello from level 1
    cat <<-EOF2
    Hello from level 2
    EOF2


In [14]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    <<'EOF'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### execution
${FD_PRJ}/sing_proj_encode_fcc.sh ls *.bed

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF

Submitted batch job 28200318


In [17]:
cat sing_heredoc.txt

Slurm Array Index: 
Time Stamp:         05-02-22+11:30:34

A.bed
B.bed

Done!
Run Time: 1 seconds


## Learn the scope of nested heredoc

https://phoenixnap.com/kb/bash-heredoc
```
Escape Characters
To avoid character interpretation, add a backslash (\) before a character:

cat << EOF
\$100
EOF

Alternatively, avoid character interpretation completely by escaping the delimiter:

cat << \EOF
$100
EOF
```
https://stackoverflow.com/questions/50151603/is-it-possible-to-have-variable-substitution-in-a-nested-heredoc

https://unix.stackexchange.com/questions/405250/passing-and-setting-variables-in-a-heredoc

https://unix.stackexchange.com/questions/169771/command-line-macros-in-comments-using-slurm

In [2]:
FOO="foo"
/bin/bash << EOF1
BAR="bar"
echo "FOO=$FOO"
echo "BAR=$BAR"
EOF1

FOO=foo
BAR=


In [29]:
FOO="foo"
/bin/bash << EOF
BAR="bar"
echo "FOO=$FOO"
echo "BAR=\$BAR"
EOF

FOO=foo
BAR=bar


In [26]:
FOO="foo"
/bin/bash << 'EOF'
BAR="bar"
echo "FOO=$FOO"
echo "BAR=$BAR"
EOF

FOO=
BAR=bar


In [28]:
FOO="foo"
/bin/bash << "EOF"
BAR="bar"
echo "FOO=$FOO"
echo "BAR=$BAR"
EOF

FOO=
BAR=bar


In [4]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sbatch_heredoc.txt \
    << 'EOF'
#!/bin/bash
ls 
EOF

Submitted batch job 28200369


In [5]:
cat sbatch_heredoc.txt

A.bed
add_starrseq_to_abc_copy.ipynb
B.bed
bedtools_coverage.ipynb
mp_benchmark.ipynb
mp_benchmark.py
R_bins_barplot.ipynb
R_slice_window.ipynb
sbatch_heredoc.txt
search_alex_code.ipynb
Sing_heredoc.ipynb
sing_heredoc.txt


In [9]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sbatch_heredoc.txt \
    << EOF1
#!/bin/bash
ls 
EOF1

Submitted batch job 28200371


In [8]:
cat sbatch_heredoc.txt

A.bed
add_starrseq_to_abc_copy.ipynb
B.bed
bedtools_coverage.ipynb
mp_benchmark.ipynb
mp_benchmark.py
R_bins_barplot.ipynb
R_slice_window.ipynb
sbatch_heredoc.txt
search_alex_code.ipynb
Sing_heredoc.ipynb
sing_heredoc.txt


In [32]:
### execution
X=10
${FD_PRJ}/sing_proj_encode_fcc.sh 10

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/sing_proj_encode_fcc.sh
/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
10


In [44]:
### execution
X=10
${FD_PRJ}/sing_proj_encode_fcc.sh "--env" ${FP_SIF}

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/sing_proj_encode_fcc.sh: illegal option -- -
Usage: hello

: 1

In [16]:
### execution
X=10
${FD_PRJ}/sing_proj_encode_fcc.sh ${FP_SIF} /bin/bash << 'EOF'
    Y=20
    echo X: ${X}
    echo Y: ${Y}
EOF

/gpfs/fs1/data/reddylab/Kuei/singularity/singularity_proj_combeffect.sif
/bin/bash
X:
Y: 20


In [13]:
### execution
X=10
singularity exec \
    --env X=${X} \
    --env Z=50 \
    ${FD_SING}/singularity_proj_combeffect.sif /bin/bash << 'EOF'
    Y=20
    echo X: ${X}
    echo Y: ${Y}
    echo Z: ${Z}
EOF

X: 10
Y: 20
Z: 50


In [24]:
### execution
X=10
${FD_PRJ}/sing_proj_encode_fcc.sh --env X=${X} ${FP_SIF} /bin/bash

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/sing_proj_encode_fcc.sh: illegal option -- -
Usage: hello

: 1

In [18]:
### execution
X=10
${FD_PRJ}/sing_proj_encode_fcc.sh \
    --env X=${X} \
    --env Z=50 \
    ${FP_SIF} /bin/bash << 'EOF'
    Y=20
    echo X: ${X}
    echo Y: ${Y}
    echo Z: ${Z}
EOF

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks/sing_proj_encode_fcc.sh: illegal option -- -
Usage: hello

: 1

In [52]:
X=10
./test.sh a b c

a
b
c


In [64]:
X=10
./test.sh -vv 10

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Proj_CombEffect_ENCODE_FCC/notebooks
PATH OF RESULTS:    /gpfs/fs1/data/reddylab/Kuei/out/proj_combeffect_encode_fcc

-vv
10


In [55]:
X=10
./test.sh --env X=${X} -c a b c

Usage: hello

: 1

In [14]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    << EOF1
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
X=10

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"


EOF1

(standard_in) 2: syntax error
Submitted batch job 28200375


In [12]:
cat sing_heredoc.txt 

Slurm Array Index: 
Time Stamp:         05-02-22+15:11:10



In [17]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    << 'EOF1'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
X=10

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### execution
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash << EOF2
    ls /mount/project/scratch/*.bed
    Y=20
    echo X: ${X}
    echo Y: \${Y}
EOF2

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF1

Submitted batch job 28200377


In [18]:
cat sing_heredoc.txt

Slurm Array Index: 
Time Stamp:         05-02-22+15:17:35

/mount/project/scratch/A.bed
/mount/project/scratch/B.bed
X: 10
Y: 20

Done!
Run Time: 1 seconds


In [19]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    << 'EOF1'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
X=10

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### execution
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash << 'EOF2'
    ls /mount/project/scratch/*.bed
    Y=20
    echo X: ${X}
    echo Y: ${Y}
EOF2

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF1

Submitted batch job 28200378


In [20]:
cat sing_heredoc.txt

Slurm Array Index: 
Time Stamp:         05-02-22+15:18:35

/mount/project/scratch/A.bed
/mount/project/scratch/B.bed
X:
Y: 20

Done!
Run Time: 1 seconds


In [21]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    << 'EOF1'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
X=10

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### execution
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash << 'EOF2'
    ls /mount/project/scratch/*.bed
    Y=20
    echo X: \${X}
    echo Y: ${Y}
EOF2

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF1

Submitted batch job 28200379


In [22]:
cat sing_heredoc.txt

Slurm Array Index: 
Time Stamp:         05-02-22+15:19:05

/mount/project/scratch/A.bed
/mount/project/scratch/B.bed
X: ${X}
Y: 20

Done!
Run Time: 1 seconds


In [ ]:
sbatch -p ${NODE} \
    --mem 2G \
    -o sing_heredoc.txt \
    << 'EOF1'
#!/bin/bash
### set directories & global variables
source ../config_duke.sh
X=10

### print start message
timer_start=`date +%s`
echo "Slurm Array Index: " ${SLURM_ARRAY_TASK_ID}
echo "Time Stamp:        " $(date +"%m-%d-%y+%T")
echo

### execution
${FD_PRJ}/sing_proj_encode_fcc.sh /bin/bash << EOF2
    ls /mount/project/scratch/*.bed
    Y=20
    echo X: ${X}
    echo Y: ${Y}
EOF2

### print end message
timer=`date +%s`
runtime=$(echo "${timer} - ${timer_start}" | bc -l)
echo
echo 'Done!'
echo "Run Time: $(displaytime ${runtime})"

EOF1

In [20]:
cat ${FD_PRJ}/sing_proj_encode_fcc.sh

#!/bin/bash

### get file path
### Stackoverflow: how-can-i-get-the-source-directory-of-a-bash-script-from-within-the-script-itself
SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" &> /dev/null && pwd )"
source ${SCRIPT_DIR}/config_duke.sh
 
### execute
singularity exec \
    -H ${PWD}:/home \
    -B ${FD_WORK}:/mount/work \
    -B ${FD_PRJ}:/mount/project \
    -B ${FD_RLAB}:/mount/reddylab \
    ${FD_SING}/singularity_proj_combeffect.sif "$@"

